# Time Series Automated Methods

## Import Libraries and Data


In [2]:
import pandas as pd
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.autogen.forecast_config import *
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.common.features.timeseries_features import *
from greykite.common.evaluation import EvaluationMetricEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results
from sklearn.metrics import mean_squared_error
from sklearn.metrics import (
    mean_absolute_error,
    root_mean_squared_error,
    mean_absolute_percentage_error,
)

t_df = pd.read_csv("../data/t.csv")
v_df = pd.read_csv("../data/v.csv")

In [3]:
t_df.head()

,timestamp,open,high,low,close,volume
0,2019-01-02,28.50,29.59,28.30,29.54,39293953
1,2019-01-03,29.52,29.81,29.21,29.58,37603571
2,2019-01-04,29.96,30.38,29.89,30.34,42072388
3,2019-01-07,30.73,31.27,30.42,30.89,41070898
4,2019-01-08,31.31,31.38,31.03,31.28,43567261


In [4]:
t_df.tail(35)

,timestamp,open,high,low,close,volume
573,2021-04-13,29.86,29.860,29.510,29.63,41385404
574,2021-04-14,29.60,29.795,29.300,29.73,31287888
575,2021-04-15,29.78,29.840,29.680,29.75,26490820
576,2021-04-16,29.91,30.030,29.840,29.95,30163100
577,2021-04-19,30.00,30.075,29.810,29.99,29788532
578,2021-04-20,29.97,30.020,29.850,29.89,27038675
579,2021-04-21,29.96,30.235,29.905,30.11,34357370
580,2021-04-22,31.30,31.890,31.150,31.36,108523112
581,2021-04-23,31.48,31.710,31.400,31.40,40369340
582,2021-04-26,31.38,31.380,30.890,30.91,45461322


## Silverkite


In [5]:
# Specify the time series data with 2021-04-30 as the training end date
metadata = MetadataParam(
    time_col="timestamp", value_col="close", freq="D", train_end_date="2021-04-30"
)
metadata

MetadataParam(anomaly_info=None, date_format=None, freq='D', time_col='timestamp', train_end_date='2021-04-30', value_col='close')

In [6]:
# Define the growth terms
growth = dict(growth_term=["linear", "quadratic", "sqrt"])
growth

{'growth_term': ['linear', 'quadratic', 'sqrt']}

In [7]:
# Define the seasonality terms
seasonality = dict(
    yearly_seasonality="auto",
    quarterly_seasonality="auto",
    monthly_seasonality="auto",
    weekly_seasonality="auto",
    daily_seasonality="auto",
)
seasonality

{'yearly_seasonality': 'auto',
 'quarterly_seasonality': 'auto',
 'monthly_seasonality': 'auto',
 'weekly_seasonality': 'auto',
 'daily_seasonality': 'auto'}

In [8]:
get_available_holiday_lookup_countries(["US"])
get_available_holidays_across_countries(
    countries=["US"], year_start=2015, year_end=2021
)

['Christmas Day',
 'Christmas Day (Observed)',
 'Columbus Day',
 'Halloween',
 'Independence Day',
 'Independence Day (Observed)',
 'Juneteenth National Independence Day',
 'Juneteenth National Independence Day (Observed)',
 'Labor Day',
 'Martin Luther King Jr. Day',
 'Memorial Day',
 "New Year's Day",
 "New Year's Day (Observed)",
 'Thanksgiving',
 'Veterans Day',
 'Veterans Day (Observed)',
 "Washington's Birthday"]

In [9]:
# Specifying events
events = dict(
    holiday_lookup_countries=["US"], holiday_pre_num_days=2, holiday_post_num_days=2
)
events

{'holiday_lookup_countries': ['US'],
 'holiday_pre_num_days': 2,
 'holiday_post_num_days': 2}

In [10]:
# Specifying changepoints
changepoints = dict(changepoints_dict=dict(method="auto"))
changepoints

{'changepoints_dict': {'method': 'auto'}}

In [11]:
# Regressors
regressors = dict(regressor_cols=["open", "high", "low", "volume"])
regressors

{'regressor_cols': ['open', 'high', 'low', 'volume']}

In [12]:
# Lagged Regressors
lagged_regressors = dict(
    lagged_regressor_dict={
        "open": "auto",
        "high": "auto",
        "low": "auto",
        "volume": "auto",
    }
)
lagged_regressors

{'lagged_regressor_dict': {'open': 'auto',
  'high': 'auto',
  'low': 'auto',
  'volume': 'auto'}}

In [13]:
# Autoregression
autoregression = dict(autoreg_dict="auto")
autoregression

{'autoreg_dict': 'auto'}

In [14]:
# Fitting algorithms
custom = dict(
    fit_algorithm_dict=[
        dict(fit_algorithm="linear"),
        dict(fit_algorithm="ridge"),
        dict(fit_algorithm="rf"),
        dict(fit_algorithm="gradient_boosting"),
    ]
)
custom

{'fit_algorithm_dict': [{'fit_algorithm': 'linear'},
  {'fit_algorithm': 'ridge'},
  {'fit_algorithm': 'rf'},
  {'fit_algorithm': 'gradient_boosting'}]}

In [15]:
# Build the model
model_components = ModelComponentsParam(
    growth=growth,
    seasonality=seasonality,
    events=events,
    changepoints=changepoints,
    regressors=regressors,
    lagged_regressors=lagged_regressors,
    autoregression=autoregression,
    custom=custom,
)

In [31]:
# Cross-validation
evaluation_period = EvaluationPeriodParam(
    cv_min_train_periods=t_df.shape[0] - 60, cv_expanding_window=True
)

# Evaluation metric
evaluation_metric = EvaluationMetricParam(
    cv_selection_metric=EvaluationMetricEnum.RootMeanSquaredError.name
)

In [36]:
# Configuration
config = ForecastConfig(
    model_template=ModelTemplateEnum.AUTO.name,
    forecast_horizon=30,
    metadata_param=metadata,
    model_components_param=model_components,
    evaluation_period_param=evaluation_period,
    evaluation_metric_param=evaluation_metric
)

In [37]:
# Forecasting
forecaster = Forecaster()
result = forecaster.run_forecast_config(df=t_df, config=config)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/greykite/common/time_properties.py:361: UserWarning:

Provided frequency 'D' does not match inferred frequency 'B'. Using 'D'.

/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/greykite/common/time_properties.py:361: UserWarning:

Provided frequency 'D' does not match inferred frequency 'B'. Using 'D'.



ValueError: 
All the 36 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
36 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/pipeline.py", line 654, in fit
    Xt = self._fit(X, y, routed_params, raw_params=params)
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/pipeline.py", line 588, in _fit
    X, fitte

In [ ]:
metadata = MetadataParam(
    time_col="timestamp",  # name of the time column
    value_col="close",  # name of the value column
    freq="D",  # "H" for hourly, "D" for daily, "W" for weekly, etc.
)

forecaster = Forecaster()
result = forecaster.run_forecast_config(
    df=t_df,
    config=ForecastConfig(
        model_template=ModelTemplateEnum.SILVERKITE.name,
        forecast_horizon=30,  # forecasts 365 steps ahead
        coverage=0.95,  # 95% prediction intervals
        metadata_param=metadata,
    ),
)

# %%
# For a detailed documentation about the output from
# :py:meth:`~greykite.framework.templates.forecaster.Forecaster.run_forecast_config`,
# see :doc:`/pages/stepbystep/0500_output`. Here we could plot the forecast.

forecast = result.forecast
fig = forecast.plot()

Fitting 3 folds for each of 1 candidates, totalling 3 fits


/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/greykite/common/time_properties.py:361: UserWarning:

Provided frequency 'D' does not match inferred frequency 'B'. Using 'D'.

/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/greykite/common/time_properties.py:419: UserWarning:

`train_end_date` is not provided, or close column of the provided time series contains null values at the end, or the input `train_end_date` is beyond the last timestamp available. Setting `train_end_date` to the last timestamp with a non-null value (2021-06-01 00:00:00).

/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/greykite/common/time_properties.py:361: UserWarning:

Provided frequency 'D' does not match inferred frequency 'B'. Using 'D'.

/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/greykite/common/time_properties.py:419: UserWarning:

`train_end_date` is not provided, or close column of the provided time series contains null values at the 

ValueError: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/pipeline.py", line 654, in fit
    Xt = self._fit(X, y, routed_params, raw_params=params)
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/pipeline.py", line 588, in _fit
    X, fitted_

# Peyton Manning

In [14]:
import pandas as pd

df = pd.DataFrame(
    {
        "date": ["2020-01-08-00", "2020-01-09-00", "2020-01-10-00"],
        "sessions": [10231.0, 12309.0, 12104.0],
    }
)

In [15]:
from collections import defaultdict
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import plotly

from greykite.common.data_loader import DataLoader
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results

# Loads dataset into pandas DataFrame
dl = DataLoader()
df = dl.load_peyton_manning()

# specify dataset information
metadata = MetadataParam(
    time_col="ts",  # name of the time column ("date" in example above)
    value_col="y",  # name of the value column ("sessions" in example above)
    freq="D"  # "H" for hourly, "D" for daily, "W" for weekly, etc.
              # Any format accepted by `pandas.date_range`
)



In [16]:
from greykite.framework.templates.autogen.forecast_config import ModelComponentsParam

model_components = ModelComponentsParam(
    # Instead of using an "auto" list of algorithms, fix one explicitly.
    custom={"fit_algorithm_dict": {"fit_algorithm": "ridge"}},
    # You might also explicitly set growth to a literal value
    growth={"growth_term": "linear"},
    # If autoregression is not required, you could override with an empty dict
    autoregression={"autoreg_dict": {}},
    changepoints={"changepoints_dict": {"method": "auto"}},
    events={
        "holiday_lookup_countries": ["US"],
        "holiday_pre_num_days": 2,
        "holiday_post_num_days": 2,
    },
    regressors={"regressor_cols": []},
    lagged_regressors={"lagged_regressor_dict": {}},
    seasonality={
        "yearly_seasonality": 10,
        "quarterly_seasonality": False,
        "monthly_seasonality": False,
        "weekly_seasonality": 5,
        "daily_seasonality": False,
    },
    uncertainty=None,
)

In [17]:
from greykite.framework.templates.autogen.forecast_config import ForecastConfig

config = ForecastConfig(
    model_template=ModelTemplateEnum.SILVERKITE.name,
    forecast_horizon=365,
    coverage=0.95,
    metadata_param=MetadataParam(time_col="ts", value_col="y", freq="D"),
    model_components_param=model_components,
)
result = forecaster.run_forecast_config(df=df, config=config)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


ValueError: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/pipeline.py", line 654, in fit
    Xt = self._fit(X, y, routed_params, raw_params=params)
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/pipeline.py", line 588, in _fit
    X, fitted_

Fitting 3 folds for each of 1 candidates, totalling 3 fits


ValueError: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/pipeline.py", line 654, in fit
    Xt = self._fit(X, y, routed_params, raw_params=params)
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/pipeline.py", line 588, in _fit
    X, fitted_